Scrapy Class


In [1]:
import scrapy
# Import crawler process
from scrapy.crawler import CrawlerProcess
import re
from unidecode import unidecode 
import csv


class laprensa_spider(scrapy.Spider):
    name = 'laprensa_spider'
    global news_list

    @staticmethod
    def clean_text(text):
        text = text.strip()

        text = text.replace("\n", " ")
        text = text.replace("\t", " ")
        text = text.replace("\r", " ")
        text = ''.join(c if c == 'ñ' else unidecode(c) for c in text)
        text = re.sub(r"Siguenos en Facebook:  La Prensa Oficial  y en Twitter:  @laprensaoem.*?$", "", text)
        
        
        text = text.strip()
        regex = r"Policiaca\s+[A-Za-z¡¿\"]+.+?\s{3,}"
        text = re.sub(regex, " ", text)
        
        text = re.sub(r'\s+', ' ', text).strip()
        text = text.lower()
        return text
    
    headers = ["url", "date", "article"]
    with open("text_data.csv", mode="w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(headers)
        


    def start_requests(self):
        urls = pages
        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse)

    def parse(self, response):
        links = response.xpath('//h4[@class="title "]/a/@href').extract()
        for link in links:
            yield response.follow(url=link, callback=self.parse2)

    def parse2(self, response):
        # Extract the title and text
        title = response.css('h1.title::text').extract_first().strip()
        text = response.xpath('//div[contains(@id,"content-body")]/p//text()').extract()
        # Join the text
        text = ' '.join(text).strip()
        # Construct the new and clean
        article = title + ": " + text
        
        article = laprensa_spider.clean_text(article)
        
        
        
        # date
        date = response.xpath('//p[@class="published-date"]/text()').extract_first()
        date = date.lstrip("\n\xa0 / ")
        date = date.rstrip("\n")

        # Saving in csv
        row = [response.url, date, article]
        with open("text_data.csv", mode="a", newline="") as f:
            writer = csv.writer(f)
            writer.writerow(row)

Scrapy the web

In [2]:
pages = []
for i in range(2):
    page = 'https://www.la-prensa.com.mx/barebone/wf.template/config.default.master.withoutgroupcount?q=alcald%C3%ADa&section=2403&page=' + str(i + 1) + '&contentType=story&sort=lastmodifieddate&order=DESC'
    pages.append(page)



news_list = []
process = CrawlerProcess()
process.crawl(laprensa_spider)
process.start()

2024-11-06 12:53:03 [scrapy.utils.log] INFO: Scrapy 2.11.2 started (bot: scrapybot)
2024-11-06 12:53:03 [scrapy.utils.log] INFO: Versions: lxml 5.3.0.0, libxml2 2.12.9, cssselect 1.2.0, parsel 1.9.1, w3lib 2.2.1, Twisted 24.7.0, Python 3.10.12 (main, Sep 11 2024, 15:47:36) [GCC 11.4.0], pyOpenSSL 24.2.1 (OpenSSL 3.3.2 3 Sep 2024), cryptography 43.0.1, Platform Linux-5.15.153.1-microsoft-standard-WSL2-x86_64-with-glibc2.35
2024-11-06 12:53:03 [scrapy.addons] INFO: Enabled addons:
[]
2024-11-06 12:53:03 [py.warnings] WARNING: /home/christian/taller/crime-prediction-monitoring-cdmx/.venv/lib/python3.10/site-packages/scrapy/utils/request.py:254: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility reasons, but it will change in a f

In [5]:
import pandas as pd
pandas_data = pd.read_csv("text_data.csv")

pandas_data.head()

,url,date,article
0,https://www.la-prensa.com.mx/policiaca/embiste...,lunes 4 de noviembre de 2024,embisten y matan a persona en situacion de cal...
1,https://www.la-prensa.com.mx/policiaca/incendi...,domingo 3 de noviembre de 2024,incendio consume diez locales en plaza del cen...
2,https://www.la-prensa.com.mx/policiaca/balacer...,lunes 4 de noviembre de 2024,"atoran a dos presuntos sicarios de ""el memin"",..."
3,https://www.la-prensa.com.mx/policiaca/vincula...,lunes 4 de noviembre de 2024,"vinculan a proceso a miguel ""n"" por feminicidi..."
4,https://www.la-prensa.com.mx/policiaca/detecta...,lunes 4 de noviembre de 2024,detectan irregularidades en administracion ant...


In [4]:
pandas_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   url      20 non-null     object
 1   date     20 non-null     object
 2   article  20 non-null     object
dtypes: object(3)
memory usage: 608.0+ bytes
